In [ ]:
!pip install langchain langchain-community rank_bm25 pypdf unstructured chromadb
!pip install unstructured['pdf'] unstructured
!apt-get install poppler-utils
!apt-get install -y tesseract-ocr
!apt-get install -y libtesseract-dev
!pip install pytesseract
!pip install bitsandbytes accelerate peft safetensors sentencepiece
!pip install nltk
!pip install -U sentence-transformers

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libtesseract-dev is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings

from langchain.llms import HuggingFaceHub
import torch
from transformers import ( AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline, )
from langchain import HuggingFacePipeline
from sentence_transformers import SentenceTransformer

from langchain.retrievers import BM25Retriever, EnsembleRetriever

import os
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from langchain.document_loaders import PyPDFLoader
doc_path = "/content/drive/MyDrive/rueckholung.pdf"
loader = PyPDFLoader(doc_path)
pages = loader.load_and_split()

In [ ]:
import nltk
from nltk.stem.snowball import SnowballStemmer

nltk.download('stopwords')

def preprocess(text):

    # Tokenize the text
    tokens = nltk.word_tokenize(text)

    # lowercase the tokens
    tokens = [token.lower() for token in tokens]

    # stemming
    stemmer = SnowballStemmer('german', ignore_stopwords=True)
    tokens = [stemmer.stem(token) for token in tokens]

    return tokens

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from langchain_core.documents import Document

#text = pages[12].page_content
text1 = "Wandern im Schweizer Jura. Die schönsten Routen und besten Einkehrmöglichkeiten"
text2 = "So verschönern Sie Ihre Wände: Wandschmuck und die neusten Tapetenmuster"

preprocessed_text1 = preprocess(text1)
print(preprocessed_text1)

preprocessed_text2 = preprocess(text2)
print(preprocessed_text2)

keyword_retriever = BM25Retriever.from_documents(pages, preprocess_func=preprocess, k=5)

['wand', 'im', 'schweiz', 'jura', '.', 'die', 'schon', 'rout', 'und', 'best', 'einkehrmog']
['so', 'verschon', 'sie', 'ihre', 'wand', ':', 'wandschmuck', 'und', 'die', 'neust', 'tapetenmust']
